(유형1) 정규화

In [100]:
import pandas as pd
import numpy as np

path = 'C:/Users/KKK/Desktop/[Dataset] 작업형 제1유형/mtcars.csv'

# Get Dataframe
df = pd.read_csv(path, encoding='cp949')
subdf = df['qsec']
x = subdf.values.reshape(-1,1)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

y=scaler.fit_transform(x)

print((y>0.5).sum)
for i in y:
    if i >0.5:
        print(i)

<built-in method sum of numpy.ndarray object at 0x000001E7CA526B70>
[0.58809524]
[0.68095238]
[0.6547619]
[1.]
[0.52380952]
[0.59166667]
[0.64285714]
[0.65595238]
[0.52380952]


(유형 1) 가설 검정 t-test

In [124]:
from scipy import stats
fast = df[df['qsec']<=20]
slow = df[df['qsec']>20]
t_test = stats.ttest_ind(fast['qsec'], slow['qsec'])

print(t_test)


Ttest_indResult(statistic=-3.942497394147243, pvalue=0.00044711126996750043)


(유형 2) 풀이 1

In [130]:
from scipy.stats import chisquare

result = chisquare(df['qsec'], f_exp=df['mpg'])
print(result)

Power_divergenceResult(statistic=46.018193092876174, pvalue=0.040349220148313714)


In [132]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer # 왜 썻을까?

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import roc_auc_score

path = 'C:/Users/KKK/Desktop/[Dataset] 작업형 제2유형/'
X_train_path = path +'X_train.csv'
X_test_path = path + 'X_test.csv'
y_train_path = path + 'y_train.csv'
y_test_path = path + '1234.csv'

# Get Dataframe
X_train = pd.read_csv(X_train_path, encoding='cp949')
X_train = X_train.iloc[:, 1:]

X_test = pd.read_csv(X_test_path , encoding='cp949')
#cust id 먼저
X_test_id = X_test.iloc[:,0]
X_test = X_test.iloc[:, 1:]

y_train = pd.read_csv(y_train_path)
y_train = y_train.iloc[:,-1]



In [133]:
LabelEncoder
X_train.loc[:, ['주구매상품','주구매지점']] = \
    X_train.loc[:, ['주구매상품','주구매지점']].apply(LabelEncoder().fit_transform)
X_train.loc[:, ['환불금액']] = \
    X_train.loc[:, ['환불금액']].fillna(0)

X_test.loc[:, ['주구매상품','주구매지점']] = \
    X_test.loc[:, ['주구매상품','주구매지점']].apply(LabelEncoder().fit_transform)
X_test.loc[:, ['환불금액']] = \
    X_test.loc[:, ['환불금액']].fillna(0)

# OneHotEncoder

# X_train = pd.get_dummies( X_train.loc[:, ['주구매상품']])


IQR 이상치 제거

In [54]:
# def get_outlier(df=None, column=None, weight=1.5):
#   # target 값과 상관관계가 높은 열을 우선적으로 진행
#   quantile_25 = np.percentile(df[column].values, 25)
#   quantile_75 = np.percentile(df[column].values, 75)

#   IQR = quantile_75 - quantile_25
#   IQR_weight = IQR*weight
  
#   lowest = quantile_25 - IQR_weight
#   highest = quantile_75 + IQR_weight
  
#   outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
#   return outlier_idx

# outlier_idx = get_outlier(df=X_train, column='총구매액', weight=1.5)
# X_train.drop(outlier_idx, axis=0, inplace=True)
# print(X_train)

          총구매액     최대구매액       환불금액  주구매상품  ...  내점일수   내점당구매건수    주말방문비율  구매주기
0     68282840  11264000  6860000.0      5  ...    19  3.894737  0.527027    17
1      2136000   2136000   300000.0     21  ...     2  1.500000  0.000000     1
2      3197000   1639000        0.0      6  ...     2  2.000000  0.000000     1
3     16077620   4935000        0.0      5  ...    18  2.444444  0.318182    16
4     29050000  24000000        0.0     15  ...     2  1.500000  0.000000    85
...        ...       ...        ...    ...  ...   ...       ...       ...   ...
3494  95835000  81650000        0.0     17  ...     7  1.285714  0.111111    34
3495   3175200   3042900        0.0      3  ...     1  2.000000  1.000000     0
3496  29628600   7200000  6049600.0     22  ...     8  1.625000  0.461538    40
3497     75000     75000        0.0     32  ...     1  1.000000  0.000000     0
3498   1875000   1000000        0.0     41  ...     2  1.000000  0.000000    39

[3144 rows x 9 columns]


In [75]:
# LR
model = LogisticRegression()
model.fit(X_train, y_train)
print('LR ROCAUC Score: ', roc_auc_score(y_train, pd.DataFrame(model.predict_proba(X_train)).iloc[:,1]))

predict = model.predict_proba(X_test)
predict = pd.DataFrame(predict)
predict =predict.iloc[:, 1]

# LR Model Predict
answer = pd.concat([X_test_id, predict], axis = 1)
answer.to_csv(y_test_path, index = False)

LR ROCAUC Score:  0.6352677527639534


In [45]:
# MLP
mlp = MLPClassifier(hidden_layer_sizes = (10, ), solver = 'adam', activation = 'relu', learning_rate_init = 0.001, max_iter = 500)

mlp.fit(X_train, y_train)
print('MLP ROCAUC Score', roc_auc_score(y_train, pd.DataFrame(mlp.predict_proba(X_train)).iloc[:,1]))

NN ROCAUC Score 0.576493731698899


In [49]:
#RF
rf = RandomForestClassifier(n_estimators = 100, max_depth = 5, random_state = 999)
rf.fit(X_train, y_train)
print('RF ROCAUC Score', roc_auc_score(y_train, pd.DataFrame(rf.predict_proba(X_train)).iloc[:,1]))

NN ROCAUC Score 0.7386293101528664


In [134]:
# kNN
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')

knn.fit(X_train, y_train)
print('KNN ROCAUC SCORE', roc_auc_score(y_train, pd.DataFrame(knn.predict_proba(X_train)).iloc[:,1]))

predict = pd.DataFrame(knn.predict_proba(X_test)).iloc[:,1]
print(predict)

answer = pd.concat([X_test_id, predict], axis = 1)
answer.to_csv(y_test_path, index = False)


KNN ROCAUC SCORE 0.78464579366935
0       0.0
1       0.2
2       0.2
3       0.4
4       0.0
       ... 
2477    0.8
2478    0.4
2479    0.4
2480    0.2
2481    1.0
Name: 1, Length: 2482, dtype: float64
